In [1]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append("../")

import pandas as pd
import plotly.express as px
from tqdm.auto import tqdm
import timeit

from evobench.continuous.cec2013lsgo import (
    F1, F2, F3, F4, F5, F6, F7, F8, F9, F10, F11, F12, F13, F14, F15
)

In [2]:
pop_sizes = [1e3, 1e4, 2e4, 5e4, 1e5]
benchmarks = [
    F1(), F2(), F3(), F4(), F5(), F6(), F7(), F8(), F9(),F10(), F11(), F12(), F13(),
    F14(), F15()
]
repetitions = 10
stats = []

In [3]:
for benchmark in tqdm(benchmarks):
    for pop_size in pop_sizes:
        for repetition in range(repetitions):
            population = benchmark.initialize_population(pop_size)
            start = timeit.default_timer()
            benchmark.evaluate_population(population)
            end = timeit.default_timer()
            dt = end - start
            stats.append([benchmark.__class__.__name__, pop_size, dt])

100%|██████████| 15/15 [1:43:50<00:00, 415.34s/it]


In [5]:
stats = pd.DataFrame(stats, columns=["benchmark", "pop_size", "dt"])
stats = stats.groupby(["benchmark", "pop_size"]).agg(["mean", "max", "min"])
stats["margin"] = stats.dt["max"] - stats.dt["min"]
stats = stats.reset_index()

In [24]:
fig = px.line(
    x=["{:.0e}".format(pop_size) for pop_size in stats.pop_size.values],
    y=stats.dt["mean"],
    color=stats.benchmark,
    error_y=stats.margin,
    labels={
        "y": "dt[s]",
        "x": "pop_size",
        "color": "",
    }
)

In [25]:
fig